# Entrenamiento de audio con corrupciones
Hecho por:
* Ammi Beltrán
* Fernanda Borja
* Luciano Vidal

**Dirección del dataset**

In [1]:
DATA = "C:\\audio_dataset\\UrbanSound8K"

## Librerías

In [2]:
# Misc
import os
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
# Locals
import audio.audio_corrupts as corrupt
import audio.audio_model as audiotrain
import audio.audio_data as dataudio
# Torch
import torch
import torchaudio
from torch import nn
from torch.utils.data import DataLoader

## Dataset

In [3]:
np.random.seed(0)
file = os.path.join(DATA, "metadata", "UrbanSound8K.csv")
df = pd.read_csv(file)
df['relative_path'] = '\\audio' + '\\fold' + df['fold'].astype(str) + '\\' + df['slice_file_name'].astype(str)
df['corrupt'] = np.random.rand(len(df))
df = df[['relative_path', 'classID', 'corrupt']]
df.head()

,relative_path,classID,corrupt
0,\audio\fold5\100032-3-0-0.wav,3,0.548814
1,\audio\fold5\100263-2-0-117.wav,2,0.715189
2,\audio\fold5\100263-2-0-121.wav,2,0.602763
3,\audio\fold5\100263-2-0-126.wav,2,0.544883
4,\audio\fold5\100263-2-0-137.wav,2,0.423655


In [4]:
# from torch.utils.data import random_split
from sklearn.model_selection import train_test_split
num_items = len(df)
num_train = round(num_items * 0.8)
num_val = num_items - num_train
train, val = train_test_split(df, test_size=0.2,random_state=420)
train, val = train.reset_index(drop = True), val.reset_index(drop = True) 
# generator1 = torch.Generator().manual_seed(420)
# train, val = random_split(df, [num_train, num_val], generator=generator1)
print(val.head())
train_ds = dataudio.SoundDS(train, DATA, transform = True)
val_ds = dataudio.SoundDS(val, DATA, transform = False)
# print(len(train_ds))
# print(val_ds[1])
#
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=250, shuffle=True)
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=250, shuffle=False)

                     relative_path  classID   corrupt
0     \audio\fold5\17578-5-0-2.wav        5  0.646628
1  \audio\fold1\193698-2-0-114.wav        2  0.479139
2     \audio\fold6\66623-4-0-2.wav        4  0.367986
3      \audio\fold1\9031-3-2-0.wav        3  0.048076
4    \audio\fold7\168846-5-1-4.wav        5  0.095380


In [5]:
model = audiotrain.AudioModel()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
train = audiotrain.train(model, 100, train_dl, val_dl, criterion, optimizer, state = None, name = "clean", dir = "train_params\\clean", device = "cuda")

Epoch nro 1/100
Iter: 1/28, Loss:2.2935309410095215
Iter: 2/28, Loss:2.256622552871704
Iter: 3/28, Loss:2.2348461151123047
Iter: 4/28, Loss:2.1919450759887695
Iter: 5/28, Loss:2.141547918319702
Iter: 6/28, Loss:2.1219770908355713
Iter: 7/28, Loss:2.091522455215454
Iter: 8/28, Loss:2.068723440170288
Iter: 9/28, Loss:2.00533127784729
Iter: 10/28, Loss:2.0045390129089355
Iter: 11/28, Loss:2.0010411739349365
Iter: 12/28, Loss:1.9922263622283936
Iter: 13/28, Loss:1.931702971458435
Iter: 14/28, Loss:1.89935302734375
Iter: 15/28, Loss:1.8591936826705933
Iter: 16/28, Loss:1.8746051788330078
Iter: 17/28, Loss:1.8798503875732422
Iter: 18/28, Loss:1.8874129056930542
Iter: 19/28, Loss:1.8391063213348389
Iter: 20/28, Loss:1.7988874912261963
Iter: 21/28, Loss:1.7871270179748535
Iter: 22/28, Loss:1.783657431602478
Iter: 23/28, Loss:1.7680844068527222
Iter: 24/28, Loss:1.7282087802886963
Iter: 25/28, Loss:1.825332760810852
Iter: 26/28, Loss:1.869458794593811
Iter: 27/28, Loss:1.6730270385742188
Iter: 